### Getting to know the nonlinear system features of the Python control systems library

Let's see what the control system library has to offer!

It seems that it can do more than I initially thought.

For instance, the `forced_reponse` is in fact only usable for linear functions.
This prevents it to be used as an excitation function for any connection of systems which at least
one nonlinear systems.

However, there is a hole bunch of functions under "Input/Output systems", for instance `input_output_response`.
Together with the possibility to define arbitrary nonlinear state-space representations
via `NonlinearIOSystem` it allows for the simulation of any connection of possibly nonlinear systems.


In [75]:
import numpy as np
import matplotlib.pyplot as plt
import control as ctrl

It turns out that you can access the entries of a dictionary via the `get` function.
This allows to provide for default values.

In [ ]:
my_test_dict = {'hallo': 5.0}
print(my_test_dict['hallo'])
print(my_test_dict.get('hallo'))
print(my_test_dict.get('gibtesnicht', 8))

We start with a simple integrator which we transform from continous to discrete time via Euler forward.
Then we simulate this integrator.

We need to provide the udpate and the output function of a nonlinear discrte-time state space model.

In [ ]:
# Start with a simple integrator
#
# Euler forward discretization:
# y[k+1] = y[k] + u[k] * T_d
#
# update function: x[k+1] = x[k] + u[k] * T_d
# output function: y[k] = x[k]

def update_fun(t, x, u, params):
    xk = x[0]
    uk = u[0]
    dt = params.get('dt')
    xkp1 = xk + uk*dt
    return xkp1

def output_fun(t, x, u, params):
    xk = x[0]
    yk = xk
    return yk

params = { 'dt': 0.1 }
H_integrator = ctrl.NonlinearIOSystem(
    update_fun,
    output_fun,
    inputs=['u'],
    outputs=['y'],
    states=1,
    dt=params['dt'],
    name='Integrator',
    params=params)


print(H_integrator)


In [ ]:
# Define inputs
t_input = np.arange(0.0, 10.0, params['dt'])
# u_input = np.ones_like(t_input)
u_input = np.cos(t_input)

# Perform simulation
results = ctrl.input_output_response(H_integrator, t_input, u_input, X0=[0.0], params=params)


fig = plt.figure(1)
plt.clf()
plt.plot(t_input, u_input, label='u')
plt.plot(results.t, results.y[0,:], label='y')
plt.grid(True)
plt.legend()
plt.show()

Next, we add a discrete-time delay which we define with a transfer function (that is, not in state space).


In [ ]:
H_delay = ctrl.tf([1.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], params['dt'])
print(H_delay)

H_delay = ctrl.ss(H_delay, name='TimeDelay')

# Connect the systems
# Note the specification of the connections:
# - First the input, then the output (so you do describe the arrow backwards)
# - For the integrator, we have names without [0], as we described them like
#   that when creating the system
# - For the time delay, we have the [0] suffix, as we created the system
#   automatically via ctrl.ss
# - You can use the name of the system, so signal ports across multiple
#   systems can share the same name
H_connected = \
        ctrl.interconnect([H_integrator, H_delay],
                        connections=[['TimeDelay.u[0]', 'Integrator.y']],
                        inplist=['Integrator.u'],
                        outlist=['TimeDelay.y[0]'],
                        inputs=['u_overall'],
                        outputs=['y_overall'],
                        dt=params['dt'])

print(H_connected)

In [ ]:
# Define inputs
t_input = np.arange(0.0, 10.0, params['dt'])
# u_input = np.ones_like(t_input)
u_input = np.cos(t_input)


# Perform the simulation with the integrator and the time delay
results = ctrl.input_output_response(H_connected, t_input, u_input, X0=[0.0], params=params)


fig = plt.figure(1)
plt.clf()
plt.plot(t_input, u_input, label='u')
plt.plot(results.t, results.y[0,:], label='y')
plt.grid(True)
plt.legend()
plt.show()

Discrete-time first order lag.

In [81]:
# Define a discrete-time first-order lag system
#
# Original continuous time system
# G(s) = A / (s + B)

Discrete-time second order system.
Let's see how we can improve from Euler forward to RK4.
First: Euler forward.

In [ ]:
# Continuous-time differential equation:
# d^2 x(t)/dt^2 + x(t) = 0
#
# Continuous-time state-space:
# dx_1 / dt = x_2
# dx_2 / dt = -x_1
# y = x
#
# dx/dt = A*x
# y = C*x
#
#          [  0  1 ]
# with A = [       ]
#          [ -1  0 ]
#
# and C = [ 1, 0]
#
#

def update_fun(t, x, u, params):
    xk = x
    F = params['F']
    xkp1 = np.dot(F, xk)
    return xkp1

def output_fun(t, x, u, params):
    C = params['C']
    return np.dot(C, x)

dt = 0.1
A = np.array([[0, 1], [-1, 0]])
F = np.eye(2) + dt*A
C = np.array([[1, 0]])

params = { 'A': A, 'F': F, 'C': C, 'dt': dt }

H_osci_euler_forward = ctrl.NonlinearIOSystem(
    update_fun,
    output_fun,
    inputs=['u'],
    outputs=['y'],
    states=2,
    dt=params['dt'],
    name='Oscillator Euler forward',
    params=params
)

t_input = np.arange(0.0, 20.0, params['dt'])
u_input = np.zeros_like(t_input)

initial_condition = np.array([1.0, 0.0])
results = ctrl.input_output_response(H_osci_euler_forward, t_input, u_input, X0=initial_condition, params=params)

fig = plt.figure(1)
plt.clf()
plt.plot(results.t, results.y[0,:], label='y')
plt.grid(True)
plt.xlabel('t')
plt.legend()
plt.title('Euler forward integration')
plt.show()


Next we try RK4.

In [ ]:
# Continuous-time differential equation:
# d^2 x(t)/dt^2 + x(t) = 0
#
# Continuous-time state-space:
# dx_1 / dt = x_2
# dx_2 / dt = -x_1
# y = x
#
# dx/dt = A*x
# y = C*x
#
#          [  0  1 ]
# with A = [       ]
#          [ -1  0 ]
#
# and C = [ 1, 0]
#
#

def update_fun(t, x, u, params):
    xk = x
    A = params['A']
    dt = params['dt']
    k1 = np.dot(A, xk)
    k2 = np.dot(A, xk + 0.5*dt*k1)
    k3 = np.dot(A, xk + 0.5*dt*k2)
    k4 = np.dot(A, xk + dt*k3)
    xkp1 = xk + dt/6.0 * (k1 + 2*k2 + 2*k3 + k4)
    return xkp1

def output_fun(t, x, u, params):
    C = params['C']
    return np.dot(C, x)

dt = 0.1
A = np.array([[0, 1], [-1, 0]])
F = np.eye(2) + dt*A
C = np.array([[1, 0]])

params = { 'A': A, 'F': F, 'C': C, 'dt': dt }

H_osci_rk4 = ctrl.NonlinearIOSystem(
    update_fun,
    output_fun,
    inputs=['u'],
    outputs=['y'],
    states=2,
    dt=params['dt'],
    name='Oscillator Runge Kutta 4',
    params=params
)

t_input = np.arange(0.0, 20.0, params['dt'])
u_input = np.zeros_like(t_input)

initial_condition = np.array([1.0, 0.0])
results = ctrl.input_output_response(H_osci_rk4, t_input, u_input, X0=initial_condition, params=params)

fig = plt.figure(1)
plt.clf()
plt.plot(results.t, results.y[0,:], label='y')
plt.grid(True)
plt.xlabel('t')
plt.legend()
plt.title('RK4 integration')
plt.show()